# Calculate and save HCW and tracer mass on shelf 

It has the same content as the script but I couldn't run ir remotely because of the netcdf library load.

In [1]:
# Calculate and save HCW and tracer mass on shelf
from netCDF4 import Dataset
import numpy as np
import pandas as pd

import canyon_tools.metrics_tools as mtt 
import canyon_tools.readout_tools as rout

In [2]:
# -------------------------------------------------------------------------------------------------------------------------
def Tracer_AlongShelf(Tr,TrAdv,MaskC,rA,hFacC,drF,yin,zfin,xi,yi,nzlim):
    '''
    INPUT----------------------------------------------------------------------------------------------------------------
    Tr    : Array with concentration values for a tracer. Until this function is more general, size 19x90x360x360
    TrAdv : Array with concentration values for low diffusivity tracer. Until this function is more general, size 19x90x360x360
    MaskC : Land mask for tracer
    nzlim : The nz index under which to look for water properties
    rA    : Area of cell faces at C points (360x360)
    fFacC : Fraction of open cell (104x360x360)
    drF   : Distance between cell faces (104)
    yin   : across-shore index of shelf break
    zfin  : shelf break index + 1 
    xi    : initial profile x index
    yi    : initial profile y index
    
    OUTPUT----------------------------------------------------------------------------------------------------------------
    TrMass =  Array with the mass of tracer over the shelf in HCW [t,360] at every time output.
    Total_Tracer =  Array with the mass of tracer (m^3*[C]*l/m^3) at each x-position over the shelf [t,360] at 
                    every time output. 
    -----------------------------------------------------------------------------------------------------------------------
    '''
    maskExp = mtt.maskExpand(MaskC,TrAdv)

    TrMask=np.ma.array(TrAdv,mask=maskExp)   
    Tr1Mask=np.ma.array(Tr,mask=maskExp)   
    
    trlim1 = TrMask[0,nzlim,yi,xi]
    trlim = (trlim1)

    print('tracer limit concentration is: ',trlim)
    
    # mask cells with tracer concentration < trlim on shelf
    HighConc_Masked = np.ma.masked_less(TrMask[:,:zfin,yin:,:], trlim) 
    HighConc_Mask = HighConc_Masked.mask

    #Get volume of water of cells with relatively high concentration
    rA_exp = np.expand_dims(rA[yin:,:],0)
    drF_exp = np.expand_dims(np.expand_dims(drF[:zfin],1),1)
    rA_exp = rA_exp + np.zeros(hFacC[:zfin,yin:,:].shape)
    drF_exp = drF_exp + np.zeros(hFacC[:zfin,yin:,:].shape)

    ShelfVolume = hFacC[:zfin,yin:,:]*drF_exp*rA_exp
    ShelfVolume_exp = np.expand_dims(ShelfVolume,0)
    ShelfVolume_exp = ShelfVolume_exp + np.zeros(HighConc_Mask.shape)
   
    HighConc_CellVol = np.ma.masked_array(ShelfVolume_exp,mask = HighConc_Mask) 
    TrConc_HCW = np.ma.masked_array(Tr[:,:zfin,yin:,:],mask = HighConc_Mask) 

    MassTrHighConc =np.ma.sum(np.ma.sum(np.ma.sum(HighConc_CellVol*TrConc_HCW,axis = 1),axis=1),axis=1)

    #Get total mass of tracer on shelf
    Total_Tracer = np.ma.sum(np.ma.sum(np.ma.sum(ShelfVolume_exp*Tr1Mask[:,:zfin,yin:,:],axis = 1),axis=1),axis=1)
    
    return (MassTrHighConc, Total_Tracer)

### Input
only change vars here

In [3]:
# Load grid files
CanyonGrid='/data/kramosmu/results/TracerExperiments/REV_10TR_BF2_AST/03_dnw_pool/gridGlob.nc'
CanyonGridOut = Dataset(CanyonGrid)

CanyonGridNoC='/data/kramosmu/results/TracerExperiments/REV_10TR_BF2_AST/04_dnw_pool_noc/gridGlob.nc'
CanyonGridOutNoC = Dataset(CanyonGridNoC)

CanyonState='/data/kramosmu/results/TracerExperiments/REV_10TR_BF2_AST/03_dnw_pool/stateGlob.nc'
CanyonStateOut = Dataset(CanyonState)

# filenames ptracers canyon and no canyon cases
ptr_file = ('/data/kramosmu/results/TracerExperiments/REV_10TR_BF2_AST/03_dnw_pool/ptracersGlob.nc' )
ptr_file_NoC = ('/data/kramosmu/results/TracerExperiments/REV_10TR_BF2_AST/04_dnw_pool_noc/ptracersGlob.nc' )

# Grid params
nx = 616
ny = 360
nz = 104 # Orig. 90
nt = 19 # t dimension size 

# Indices shelf box, etc
xi = 180
yi = 50
nzlim = 29 # Bar:39, Ast:29, Orig:29
zfin = 30 # Bar:40, Ast:30, Orig:30
xfin = 615
yin =130 # Orig: 225

# Tracer keys
keys = ['Tr01','Tr02','Tr03','Tr04','Tr05','Tr06','Tr07','Tr08','Tr09','Tr10']
keyAdv = 'Tr01' # tracer used to track water (linear, Kv=10^-7)

# Experiment id
run = 'REV_10TR_BF2_AST_03'
run_NoC = 'REV_10TR_BF2_AST_04'
exp = 'REV_10TR_BF2_AST'

# Grid variables
xc = rout.getField(CanyonGrid, 'XC') # x coords tracer cells
yc = rout.getField(CanyonGrid, 'YC') # y coords tracer cells
rc = CanyonGridOut.variables['RC']
dxg = rout.getField(CanyonGrid, 'dxG') # x coords tracer cells
dyg = rout.getField(CanyonGrid, 'dyG') # y coords tracer cells

bathy = rout.getField(CanyonGrid, 'Depth')
hFacC = CanyonGridOut.variables['HFacC'][:zfin+1,:,:] 
hFacmasked = np.ma.masked_values(hFacC, 0)
MaskC = np.ma.getmask(hFacmasked)

bathyNoC = rout.getField(CanyonGridNoC, 'Depth')
hFacCNoC = CanyonGridOutNoC.variables['HFacC'][:zfin+1,:,:] 
hFacmaskedNoC = np.ma.masked_values(hFacCNoC, 0)
MaskCNoC = np.ma.getmask(hFacmaskedNoC)

rA = rout.getField(CanyonGridNoC, 'rA')

z = CanyonStateOut.variables['Z']
drF = CanyonGridOut.variables['drF']
time = CanyonStateOut.variables['T']

In [4]:
for key in keys:
    
    Tr = Dataset(ptr_file).variables[key][:,:zfin+1,:,:] 
    TrAdv = Dataset(ptr_file).variables[keyAdv][:,:zfin+1,:,:] 

    TrMassHCW, TotTrMass = Tracer_AlongShelf(Tr,TrAdv, MaskCNoC[:zfin+1,:,:], rA, 
                                             hFacCNoC, drF[:zfin+1],yin,zfin,xi,yi,nzlim)
    HCW = mtt.calc_HCW(TrAdv, MaskCNoC[:zfin+1,:,:], rA, hFacCNoC, drF[:zfin+1],nzlim=nzlim, yin=yin, 
                       xin=0, xfin=xfin, zfin=zfin, xi=xi, yi=yi)

    raw_data = {'time' : time,'HCW': HCW,'TrMassHCW': TrMassHCW,'TotTrMass':TotTrMass}
    df = pd.DataFrame(raw_data, columns = ['time' ,'HCW','TrMassHCW','TotTrMass'])

    filename1 = ('/data/kramosmu/results/TracerExperiments/%s/HCW_TrMass_%s_%s.csv' %(exp,key,run))
    df.to_csv(filename1)
    print(filename1)

tracer limit concentration is:  7.1779594
tracer limit concentration is:  7.1779594
/data/kramosmu/results/TracerExperiments/REV_10TR_BF2_AST/HCW_TrMass_Tr01_REV_10TR_BF2_AST_03.csv
tracer limit concentration is:  7.1779594
tracer limit concentration is:  7.1779594
/data/kramosmu/results/TracerExperiments/REV_10TR_BF2_AST/HCW_TrMass_Tr02_REV_10TR_BF2_AST_03.csv
tracer limit concentration is:  7.1779594
tracer limit concentration is:  7.1779594
/data/kramosmu/results/TracerExperiments/REV_10TR_BF2_AST/HCW_TrMass_Tr03_REV_10TR_BF2_AST_03.csv
tracer limit concentration is:  7.1779594
tracer limit concentration is:  7.1779594
/data/kramosmu/results/TracerExperiments/REV_10TR_BF2_AST/HCW_TrMass_Tr04_REV_10TR_BF2_AST_03.csv
tracer limit concentration is:  7.1779594
tracer limit concentration is:  7.1779594
/data/kramosmu/results/TracerExperiments/REV_10TR_BF2_AST/HCW_TrMass_Tr05_REV_10TR_BF2_AST_03.csv
tracer limit concentration is:  7.1779594
tracer limit concentration is:  7.1779594
/data/

### No canyon runs

In [5]:
for key in keys:

    TrNoC = Dataset(ptr_file_NoC).variables[key][:,:zfin+1,:,:] 
    TrNoCAdv = Dataset(ptr_file_NoC).variables[keyAdv][:,:zfin+1,:,:] 

    TrMassHCW, TotTrMass = Tracer_AlongShelf(TrNoC,TrNoCAdv, MaskCNoC, rA, hFacCNoC, 
                                             drF[:zfin+1],yin,zfin,xi,yi,nzlim)
    HCW = mtt.calc_HCW(TrNoCAdv, MaskCNoC[:zfin+1,:,:], rA, hFacCNoC, drF[:zfin+1],nzlim=nzlim, yin=yin, 
                       xin=0, xfin=xfin, zfin=zfin, xi=xi, yi=yi)

    raw_data = {'time' : time,'HCW': HCW,'TrMassHCW': TrMassHCW,'TotTrMass':TotTrMass}
    df = pd.DataFrame(raw_data, columns = ['time' ,'HCW','TrMassHCW','TotTrMass'])

    filename2 = ('/data/kramosmu/results/TracerExperiments/%s/HCW_TrMass_%s_%s.csv' %(exp,key,run_NoC))
    df.to_csv(filename2)
    print(filename2)

tracer limit concentration is:  7.1961036
tracer limit concentration is:  7.1961036
/data/kramosmu/results/TracerExperiments/REV_10TR_BF2_AST/HCW_TrMass_Tr01_REV_10TR_BF2_AST_04.csv
tracer limit concentration is:  7.1961036
tracer limit concentration is:  7.1961036
/data/kramosmu/results/TracerExperiments/REV_10TR_BF2_AST/HCW_TrMass_Tr02_REV_10TR_BF2_AST_04.csv
tracer limit concentration is:  7.1961036
tracer limit concentration is:  7.1961036
/data/kramosmu/results/TracerExperiments/REV_10TR_BF2_AST/HCW_TrMass_Tr03_REV_10TR_BF2_AST_04.csv
tracer limit concentration is:  7.1961036
tracer limit concentration is:  7.1961036
/data/kramosmu/results/TracerExperiments/REV_10TR_BF2_AST/HCW_TrMass_Tr04_REV_10TR_BF2_AST_04.csv
tracer limit concentration is:  7.1961036
tracer limit concentration is:  7.1961036
/data/kramosmu/results/TracerExperiments/REV_10TR_BF2_AST/HCW_TrMass_Tr05_REV_10TR_BF2_AST_04.csv
tracer limit concentration is:  7.1961036
tracer limit concentration is:  7.1961036
/data/

In [6]:
(12960*40)/(24*3600)

6.0